In [1]:
import time
import datetime
import locale
from pathlib import Path

import requests

In [16]:
BASE_DATA_DIR = Path("data/html/")
assert BASE_DATA_DIR.exists()

In [3]:
locale.setlocale(locale.LC_TIME, 'it_IT.UTF-8')

'it_IT.UTF-8'

In [4]:
MIN_YEAR, MAX_YEAR = 1973, 2023

In [5]:
MONTHS = [
    m for m in
    (datetime.datetime(year, month, 1) for year in range(MIN_YEAR, MAX_YEAR + 1) for month in range(1, 13))
    if m < datetime.datetime.now()
]
MONTHS[-5:]

[datetime.datetime(2023, 4, 1, 0, 0),
 datetime.datetime(2023, 5, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 7, 1, 0, 0),
 datetime.datetime(2023, 8, 1, 0, 0)]

In [49]:
successful = set()
failures = set()
for dt in MONTHS:
    year = dt.year
    month = dt.strftime("%B")
    url = f"https://www.ilmeteo.it/portale/archivio-meteo/Torino/{year}/{month}"
    resp = requests.get(url)
    if resp.status_code == 200:
        print(f"{year} - {month}: SUCCESS")
        successful.add(dt)
        path = BASE_DATA_DIR / f"{year}_{dt.month}.html"
        with path.open('w') as fp:
            fp.write(resp.text)
    else:
        print(f"{year} - {month}: ERROR")
        failures.add(dt)
    time.sleep(2)

1973 - Gennaio: SUCCESS
1973 - Febbraio: SUCCESS
1973 - Marzo: SUCCESS
1973 - Aprile: SUCCESS
1973 - Maggio: SUCCESS
1973 - Giugno: SUCCESS
1973 - Luglio: SUCCESS
1973 - Agosto: SUCCESS
1973 - Settembre: SUCCESS
1973 - Ottobre: SUCCESS
1973 - Novembre: SUCCESS
1973 - Dicembre: SUCCESS
1974 - Gennaio: SUCCESS
1974 - Febbraio: SUCCESS
1974 - Marzo: SUCCESS
1974 - Aprile: SUCCESS
1974 - Maggio: SUCCESS
1974 - Giugno: SUCCESS
1974 - Luglio: SUCCESS
1974 - Agosto: SUCCESS
1974 - Settembre: SUCCESS
1974 - Ottobre: SUCCESS
1974 - Novembre: SUCCESS
1974 - Dicembre: SUCCESS
1975 - Gennaio: SUCCESS
1975 - Febbraio: SUCCESS
1975 - Marzo: SUCCESS
1975 - Aprile: SUCCESS
1975 - Maggio: SUCCESS
1975 - Giugno: SUCCESS
1975 - Luglio: SUCCESS
1975 - Agosto: SUCCESS
1975 - Settembre: SUCCESS
1975 - Ottobre: SUCCESS
1975 - Novembre: SUCCESS
1975 - Dicembre: SUCCESS
1976 - Gennaio: SUCCESS
1976 - Febbraio: SUCCESS
1976 - Marzo: SUCCESS
1976 - Aprile: SUCCESS
1976 - Maggio: SUCCESS
1976 - Giugno: SUCCESS
19

In [9]:
assert not failures, f"Some months failed: {[f.strftime('%Y - %B') for f in failures]}"

AssertionError: Some months failed: ['2021 - Aprile']

In [15]:
MAX_RETRIES = 5
retries = {}
while len(failures) > 0:
    
    dt = failures.pop()
    retries.setdefault((dt.year, dt.month), 0)
    year, month = dt.year, dt.strftime('%B')
    
    url = f"https://www.ilmeteo.it/portale/archivio-meteo/Torino/{year}/{month}"
    resp = requests.get(url)
    if resp.status_code == 200:
        print(f"{year} - {month}: SUCCESS")
        path = BASE_DATA_DIR / f"{year}_{dt.month}.html"
        with path.open('w') as fp:
            fp.write(resp.text)
    else:
        retries[(dt.year, dt.month)] += 1
        if retries[(dt.year, dt.month)] < MAX_RETRIES:
            print(f"{year} - {month} FAILED ({retries[(dt.year, dt.month)]})")
            failures.add(dt)
        else:
            print(f"{year} - {month} MAX RETRIES REACHED ({retries[(dt.year, dt.month)]})")
    time.sleep(2)

2021 - Aprile: SUCCESS


#### Summary:

In [17]:
import pandas as pd

In [30]:
months = []
for html_path in BASE_DATA_DIR.glob("*.html"):
    year, month = html_path.stem.split("_")
    months.append(pd.Timestamp(int(year), int(month), 1))
s = pd.Series(pd.Series(months).sort_values().values)

In [31]:
s

0     1973-01-01
1     1973-02-01
2     1973-03-01
3     1973-04-01
4     1973-05-01
         ...    
603   2023-04-01
604   2023-05-01
605   2023-06-01
606   2023-07-01
607   2023-08-01
Length: 608, dtype: datetime64[ns]

In [32]:
s.is_monotonic_increasing

True